In [1]:
import numpy as np
import os
import knapsack as ks

files_ext = {0:'.in', 1:'.out'}
files = {0:'me_at_the_zoo', 1:'videos_worth_spreading', 2:'trending_today', 3:'kittens'} 

In [2]:
def load_data(id_file_to_load = 0, print_debug = True):
    print('\n***************** Loading *****************')
    # Load file
    file = open(files[id_file_to_load] + files_ext[0], 'r') 

    # Header / Video / Endpoints / request descr. / Cache
    header = file.readline().replace('\n','')
    header = np.fromstring(header, dtype=int, sep=' ')
    header_dict = {   'n_video':header[0], 'n_endpoint':header[1], 
                      'requ_des':header[2], 'n_cache':header[3],
                      'size_cache':header[4]}

    # Video size in MB
    video_size = file.readline().replace('\n','')
    video_size = np.fromstring(video_size, dtype=int, sep=' ')

    # Endpoints locations and connections
    endpoint_cache = np.zeros((header_dict['n_endpoint'], header_dict['n_cache'])) # Latency endpoint to cache
    endpoint_datacenter = np.zeros(header_dict['n_endpoint']) # Latency endpoint to datacenter
    endpoint_video = np.zeros((header_dict['n_endpoint'], header_dict['n_video'])) # Endpoints requests

    # Reding endpoint to cache latency and endpoint to data_center latency
    for id_endpoint in range(header_dict['n_endpoint']):
        endpoint = file.readline().replace('\n','').split(' ')
        endpoint_datacenter[id_endpoint] = int(endpoint[0])
        n_latency = int(endpoint[1])
        for i in range(n_latency):
            lat_cache = file.readline().replace('\n','').split(' ')
            endpoint_cache[id_endpoint, int(lat_cache[0])] = int(lat_cache[1])

    for id_requ_des in range(header_dict['requ_des']):
        requ_des = file.readline().replace('\n','').split(' ')
        endpoint_video[int(requ_des[1]), int(requ_des[0])] += int(requ_des[2])
    file.close()
    
    if print_debug:
        print('File description', '\t', header_dict)
        print('Videos sizes', '\t\tshape=', video_size.shape, ', nnz=', len(np.nonzero(video_size)[0]))
        print('Endpoint to chache', '\tshape=', endpoint_cache.shape, ', nnz=', len(np.nonzero(endpoint_cache)[0]))
        print('Endpoint to video', '\tshape=', endpoint_video.shape, ', max=', np.max(endpoint_video))
    
    return endpoint_cache, endpoint_datacenter, endpoint_video, video_size, header_dict["size_cache"]



def save_file(id_file_to_save = 0, caches_video_lists=None):
    print('\n***************** Saving *****************')
    filename = files[id_file_to_save] + files_ext[1]
    # Delete if existing
    if os.path.isfile(filename):
        os.remove(filename)
    file = open(filename, 'a') 
    
    # Write number caches
    file.write(str(len(caches_video_lists)) + '\n')

    # Write each cache
    for i, caches in enumerate(caches_video_lists):
        # Print chache id
        file.write(str(i) + ' ')
        for j, video in enumerate(caches):
            # Print video ids
            file.write(str(video))
            # Print space only if not last item
            if j != len(caches)-1:
                file.write(' ')
        file.write('\n')
    file.close()
    # Out save file
    print('... saved as \"' + filename + '\"')

# Load data
endpoint_cache, endpoint_datacenter, endpoint_video, video_size, size_cache  = load_data(id_file_to_load=0)

test_out = [[2],[3, 1],[0, 1]]
# Save data
save_file(id_file_to_save=0, caches_video_lists=test_out)


***************** Loading *****************
File description 	 {'n_cache': 10, 'n_video': 100, 'requ_des': 100, 'n_endpoint': 10, 'size_cache': 100}
Videos sizes 		shape= (100,) , nnz= 100
Endpoint to chache 	shape= (10, 10) , nnz= 32
Endpoint to video 	shape= (10, 100) , max= 3296.0

***************** Saving *****************
... saved as "me_at_the_zoo.out"


In [3]:
cache_videos_saved = np.zeros((endpoint_cache.shape[1], video_size.shape[0])) # Latency endpoint to cache
cache_videos_saved.shape

(10, 100)

In [4]:
for i in range(endpoint_cache.shape[1]):
    for j in range(endpoint_cache.shape[0]):
        if endpoint_cache[j][i] > 0:
            for k in range(video_size.shape[0]):
                if(endpoint_video[j][k] > 0):
                    # print("saved time on video", k, "with cache", i, "with endpoint", j)
                    pass
                cache_videos_saved[i][k] += (endpoint_datacenter[j] - endpoint_cache[j][i]) * endpoint_video[j][k]

In [5]:
items_value = cache_videos_saved[0].astype(int)
items_weight = video_size
items = list(zip(items_weight,items_value))
print(items)

[(20, 1302899), (11, 1069618), (50, 251635), (26, 0), (5, 0), (3, 581126), (6, 0), (32, 260326), (40, 644728), (22, 0), (4, 88192), (20, 0), (50, 0), (27, 386937), (49, 0), (44, 621291), (1, 182900), (37, 0), (35, 0), (27, 220660), (14, 0), (33, 0), (6, 0), (22, 0), (23, 0), (48, 0), (44, 163542), (14, 0), (26, 0), (9, 0), (46, 0), (44, 493155), (15, 0), (32, 0), (31, 0), (8, 0), (39, 0), (27, 0), (39, 0), (27, 0), (1, 0), (17, 0), (1, 0), (47, 0), (44, 0), (42, 0), (16, 299715), (3, 0), (44, 0), (48, 0), (5, 0), (25, 0), (4, 0), (39, 0), (39, 0), (7, 0), (24, 0), (28, 0), (14, 0), (44, 0), (22, 0), (11, 0), (27, 0), (37, 0), (11, 0), (16, 780618), (50, 0), (33, 0), (22, 0), (26, 0), (7, 0), (12, 0), (17, 0), (30, 0), (12, 0), (12, 0), (4, 0), (32, 0), (12, 0), (46, 0), (43, 0), (4, 0), (12, 212400), (34, 0), (11, 0), (7, 0), (47, 0), (29, 0), (24, 0), (40, 204633), (41, 0), (10, 0), (5, 0), (22, 0), (22, 0), (24, 0), (37, 0), (34, 0), (50, 0), (5, 650796)]


In [6]:
solution, w, v = ks.solve(items, size_cache)

In [11]:
if solution != []:
    
    print("solution of weight %d, value %d found:" % (w,v))
    print(", ".join(map(lambda t: str(t), solution)))
else:
    # this should never happen (and hasen't as far as we're concerned)
    print('no solution found(!).')
    
print(solution)

solution of weight 100, value 5300877 found:
(5, 650796), (16, 780618), (1, 182900), (4, 88192), (40, 644728), (3, 581126), (11, 1069618), (20, 1302899)
[(5, 650796), (16, 780618), (1, 182900), (4, 88192), (40, 644728), (3, 581126), (11, 1069618), (20, 1302899)]


In [10]:
# Get feature for all caches
res_tot = []
for i in range(cache_videos_saved.shape[0]):
    videos = []
    # Get items
    items_value = cache_videos_saved[i].astype(int)
    items_weight = video_size
    items = list(zip(items_weight,items_value))
    # Solve it
    solution, w, v = ks.solve(items, size_cache)
    res_tot.append()

[(20, 1302899), (11, 1069618), (50, 251635), (26, 0), (5, 0), (3, 581126), (6, 0), (32, 260326), (40, 644728), (22, 0), (4, 88192), (20, 0), (50, 0), (27, 386937), (49, 0), (44, 621291), (1, 182900), (37, 0), (35, 0), (27, 220660), (14, 0), (33, 0), (6, 0), (22, 0), (23, 0), (48, 0), (44, 163542), (14, 0), (26, 0), (9, 0), (46, 0), (44, 493155), (15, 0), (32, 0), (31, 0), (8, 0), (39, 0), (27, 0), (39, 0), (27, 0), (1, 0), (17, 0), (1, 0), (47, 0), (44, 0), (42, 0), (16, 299715), (3, 0), (44, 0), (48, 0), (5, 0), (25, 0), (4, 0), (39, 0), (39, 0), (7, 0), (24, 0), (28, 0), (14, 0), (44, 0), (22, 0), (11, 0), (27, 0), (37, 0), (11, 0), (16, 780618), (50, 0), (33, 0), (22, 0), (26, 0), (7, 0), (12, 0), (17, 0), (30, 0), (12, 0), (12, 0), (4, 0), (32, 0), (12, 0), (46, 0), (43, 0), (4, 0), (12, 212400), (34, 0), (11, 0), (7, 0), (47, 0), (29, 0), (24, 0), (40, 204633), (41, 0), (10, 0), (5, 0), (22, 0), (22, 0), (24, 0), (37, 0), (34, 0), (50, 0), (5, 650796)]
[(20, 3669138), (11, 1346058